In [ ]:
import pulp
import pandas as pd
import numpy as np
import re
from itertools import compress
import json

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

Function to read a json file

In [ ]:
def read_model_json(path_to_model):
    with open(path_to_model, "r") as input_file:
        prob_json = json.load(input_file)
    return prob_json

Function to write a json file

In [ ]:
#currently, deprecated
def write_json_model(prob_json, path_to_model, json_encoder = NpEncoder):
    with open(path_to_model, "w") as output_file:
        json.dump(prob_json, output_file, cls = json_encoder)

A function identifying conflicting constraints for the interim infeasibile solution, determining if the conflicting constraint is a hard constraint, iterating over the variables in the hard constraints and extract GPI from the variable

In [ ]:
def conflicting_gpi(prob):
    cols = ['gpi', 'ndc', 'client', 'client2', 'breakout', 'measurement', 'region', 'chain', 'subchain']
    suspect_gpi_df = pd.DataFrame(columns = cols)
    for c in prob.constraints.values():
        if not c.valid(0):
            ##get variables objects from suspect constraints
            constr_vars = c.toDict()['coefficients']
            ##get variable names
            constr_vars_names = [var['name'] for var in constr_vars]

            ##check if the constraint is a soft constraint;
            ##currently, soft constraints have lambda_ or sv_ variables
            ##any constraint that only includes P_ variables is a hard constraint
            suspect_constr = [name.startswith("P_") for name in constr_vars_names]
            ##if all the variables in the constraint were of types P_;
            if all(suspect_constr):
                ##extract GPI, NDC, CLIENT, BREAKOUT, MEASUREMENT, REGION, CHAIN from constraint variables
                price_vars = [price.replace('NONPREF_OTH', 'NONPREF-OTH').split('_')[1:] for price in constr_vars_names]
                suspect_gpi_df = suspect_gpi_df.append(pd.DataFrame(price_vars, columns = cols))
    return suspect_gpi_df

A function identifying decision variables that are set out of bounds in the interim feasibile solution, determining if that variable is a price variable and not a "soft cap variable"

In [ ]:
def violating_gpi(prob):
    cols = ['gpi', 'ndc', 'client', 'client2', 'breakout', 'measurement', 'region', 'chain', 'subchain']
    suspect_gpi_df = pd.DataFrame(columns = cols)
    for v in prob.variables():
        if not v.valid(0):
            ##get violating variable names
            ##check if they are price variables and not SV_ or lambda_ types
            if v.name.startswith("P_"):
                var_row = v.name.replace('NONPREF_OTH', 'NONPREF-OTH').split('_')[1:]
                suspect_gpi_df = suspect_gpi_df.append(pd.DataFrame([var_row], columns = cols))
    return suspect_gpi_df

Function to remove hard constraints from the problem json model given a list of suspected GPIs. Any hard constraints that has a variable corresponding to one of the suspected GPIs is removed from the problem. At the same time, a dictionary of those constraints and the suspected GPIs is saved for later use

In [ ]:
def rm_gpi_from_constr(prob_json, suspect_gpi_df):
    costr_lst = prob_json['constraints'].copy()
    gpi_lst = suspect_gpi_df.gpi.unique()
    constr_gpi_dict = {gpi: [] for gpi in gpi_lst}
    for constr in prob_json['constraints']:
        constr_var_lst = [var['name'] for var in constr['coefficients']]
        hard_constr = [name.startswith("P_") for name in constr_var_lst]
        if all(hard_constr):
            for gpi in gpi_lst:
                if any(gpi in var for var in constr_var_lst):
                    costr_lst.remove(constr)
                    constr_gpi_dict[gpi].append(constr)
                    break
    return costr_lst, constr_gpi_dict

Function to remove variables corresponding to suspected GPIs from the problem definitions

In [ ]:
#remove problematic GPIs from variables
#currently, deprecated
def rm_gpi_from_var(prob_json, var_to_rm_lst):
    var_lst = prob_json['variables'].copy()
    for var in prob_json['variables']:
        if var['name'] in var_to_rm_lst:
            var_lst.remove(var)
    return var_lst

Function to remove variables corresponding to suspected GPIs from the objective function

In [ ]:
#remove problematic GPIs from the objective
#currently, deprecated
def rm_gpi_from_obj(prob_json, var_to_rm_lst):
    obj_var_lst = prob_json['objective']['coefficients'].copy()
    for var in prob_json['objective']['coefficients']:
        if var in var_to_rm_lst:
            obj_var_lst.remove(var)
    return obj_var_lst

In the following cell, an infeasible problem will be subjected to a while loop. In the loop, a list of suspect GPIs is generated for the infeasible problem. All the hard constraints are then removed from the problem, and the problem is solved again. If the reduced problem is optimal, hard constraints that are removed are added back one by one to obtain a minimal list of problematic GPIs. If the reduced problem turned out to be infeasible again, a new list of suspected GPIs is generated. If the loop could not reach optimality or produce new suspected GPIs, an error is shown!

**Important NOTE**: The required json files containing the model are produced by CPMO in the same folder as the .lp files.

To submit multiple clients at once, a list of the clients should be created as below. The json files should be gathered into a single folder with the path submitted as `orig_path_to_model`. The name of the json file follows the convention `<client_code>_total_prob.json`.

For each client, an exclusion file will be generated with the name convention `infeasible_exclusion_gpis_<client>.csv`. Once these files are generated they should be put into the Input file folder while running the model. The name does not need to be altered unless the `INFEASIBLE_EXCLUSION_FILE` value is changed from its default in CPMO_parameters.py

In [ ]:
inf_clients = [3266,3502,3782,3876]
for client in inf_clients:
    orig_path_to_model = '/home/jupyter/inf_jsons/{0}_total_prob.json'.format(client)
    cols = ['gpi', 'ndc', 'client', 'client2', 'breakout', 'measurement', 'region', 'chain', 'subchain']
    suspect_gpi_df = pd.DataFrame(columns = cols)

    #run the lp for first time
    var, prob = pulp.LpProblem.from_json(orig_path_to_model)
    prob.solve()
    print(f"status: {prob.status}, {pulp.LpStatus[prob.status]}")
    print(f"objective: {prob.objective.value()}")
    status = pulp.LpStatus[prob.status]
    orig_prob_json = read_model_json(orig_path_to_model)
    prob_json = orig_prob_json.copy()

    while status == "Infeasible":
        #create initial df of suspect GPIs
        new_found_gpi_df = pd.concat([conflicting_gpi(prob), violating_gpi(prob)])
        new_found_gpi_df.drop_duplicates(inplace = True, ignore_index = True)
        if new_found_gpi_df.empty:
            #infeasibility lies in the interaction of different a mix of soft and hard constraints
            #this needs further research because something like this should not happen!
            #or there are other hard constraints which are not identified in this notebook
            print("Error!: No more GPIs are suspected but the infeasibility issue persists.")
            break
        suspect_gpi_df = suspect_gpi_df.append(new_found_gpi_df)

        print(f"unique number of new suspect GPIs found: {len(new_found_gpi_df.gpi.unique())}")
        suspect_gpi_master_lst = list(suspect_gpi_df.gpi.unique())

        print(f"Identifying hard constraints to be removed from the original problem...")    
        constr_lst, constr_gpi_dict = rm_gpi_from_constr(prob_json,
                                                         suspect_gpi_df.loc[suspect_gpi_df.gpi.isin(suspect_gpi_master_lst), ])

        print(f"Solving the reduced problem...")
        prob_json['constraints'] = constr_lst
        var, prob = pulp.LpProblem.from_dict(prob_json)
        prob.solve()
        print(f"status: {prob.status}, {pulp.LpStatus[prob.status]}")
        print(f"objective: {prob.objective.value()}")
        status = pulp.LpStatus[prob.status]

        if pulp.LpStatus[prob.status] == "Optimal":
            safe_gpi = []
            unsafe_gpi = []
            print("Add back hard constraint corresponding to suspect GPIs one by one...")
            for gpi in suspect_gpi_master_lst:
                constr_lst.extend(constr_gpi_dict[gpi])
                prob_json['constraints'] = constr_lst
                new_var, new_prob = pulp.LpProblem.from_dict(prob_json)
                new_prob.solve()
                if pulp.LpStatus[new_prob.status] == "Optimal":
                    safe_gpi.append(gpi)
                    print(f"{gpi} from the suspected list is safe")
                else:
                    unsafe_gpi.append(gpi)
                    print(f"{gpi} from the suspected list is unsafe")
                    for constr in constr_gpi_dict[gpi]:
                        constr_lst.remove(constr)
            print(f"List of unsafe GPIs: {unsafe_gpi}")

    #Write problem gpis to exclusion file
    region = 'ALL'
    gpi_exclusion = pd.DataFrame(columns = ['CLIENT', 'REGION', 'GPI'])
    gpi_exclusion['GPI'] = unsafe_gpi
    gpi_exclusion['CLIENT'] = client
    gpi_exclusion['REGION'] = region
    gpi_exclusion.to_csv('/home/jupyter/infeasible_exclusions/infeasible_exclusion_gpis_{0}.csv'.format(client), index=False)